In [3]:
from abc_utils import *
import pandas as pd
from hmmlearn import hmm
import numpy as np
from sklearn.metrics import accuracy_score

In [40]:
train_set, train_lengths, val_set, _ = load_datasets()

In [ ]:
def load_song_subset(train_set, train_lengths, indices):
    """
    Create a subset of the train"""
    end_positions = np.cumsum(train_lengths)
    positions = np.insert(end_positions, 0, np.array([0]))
    songs = []
    for i in indices:
        song = train_set.iloc[positions[i] : positions[i+1]]
        songs.append(song)
    # return songs and lengths
    return pd.concat(songs)

total_songs = 10
song_indices = list(range(total_songs))
songs = load_song_subset(train_set, train_lengths, song_indices)

In [42]:
def ffill_obs(melody_obs: np.ndarray, unique_obs: dict) -> np.ndarray:
    # make a smaller array out of the unique observations
    possible_obs = list(set(unique_obs.flatten()))

    df_melody_obs = pd.Series(melody_obs)
    df_melody_obs[~df_melody_obs.isin(possible_obs)] = np.nan

    # fill forward first to fill all the holes
    df_melody_obs.ffill(inplace=True)

    # then fill backward to catch the case where
    # the beginning is empty
    df_melody_obs.bfill(inplace=True)

    return df_melody_obs.values.flatten()

In [45]:
def chord_accuracy(full_pred: np.array, true_states: np.array, num_chords: int=None, num_notes: int=None):
    '''
    Given the predicted matrix of states, compute the misclassification rate compared with the true_observations.
    Could be edited in the future to also compute the accuracy of our predicted note sequence.
    '''
    # check to make sure these are specified correctly
    if num_chords is None:
        raise ValueError("num_chords must be specified")
    if num_notes is None:
        raise ValueError("num_notes must be specified")
    
    # obtain the actual predicted chords 
    pred_chords = full_pred[:, num_chords-1]
    true_chords = true_states[:len(pred_chords), num_chords-1]

    # obtain the accuracy
    chord_acc = accuracy_score(true_chords, pred_chords)

    # # obtain the actual predicted notes from the state
    # if num_notes != 0: 
    #     pred_notes = full_pred[:, -1]
    #     true_notes = true_states[:len(pred_notes), -1]

    #     # obtain the accuracy
    #     note_acc = accuracy_score(true_notes, pred_notes)
    # else:
    #     note_acc = None
    return chord_acc

In [ ]:
def fit_model(train_set: pd.DataFrame, train_lengths: pd.Series, num_chords: int=1, num_notes: int=0, subset: bool=False, indices=None):
    """ 
    Takes in the train set and parameters for the state space and returns the trained model, along with all of the dictionaries needed to decode the model as a tuple.

    To train on a smaller subset of the full train set, use the subset argument and pass in the indices needed. Uses the load_song_subset function.
    """
    # check if we want to do a subset of the full train set; if so, perform it
    if subset:
        # check that indices are specified; raise and error if not
        if indices is None:
            raise ValueError("Indices must be specified if subset=True")
        train_set = load_song_subset(train_set, train_lengths, indices)

    # obtain the states and observations from the songs
    true_states, true_observations = dataframe_to_states(train_set, num_chords, num_notes)

    # create the transition matrices for the model
    transition_matrix, emission_probs, unique_states, unique_obs, states_to_index, observation_to_index = states_to_transition(true_states, true_observations)

    # now initialize the model and set the matrices for it
    model = hmm.CategoricalHMM(n_components=transition_matrix.shape[0], init_params='')
    model.transmat_ = transition_matrix.T
    model.emissionprob_ = emission_probs.T
    start_probs = np.ones(transition_matrix.shape[0]) / transition_matrix.shape[0]      # use a uniform distribution on the states for the initial probabilities
    model.startprob_ = start_probs

    # return the model,  the dictionaries
    return model, (unique_states, unique_obs, states_to_index, observation_to_index)

In [95]:
def predict_states(model: hmm.CategoricalHMM, all_dicts: tuple, observation: np.ndarray):
    """
    Uses the model to decode an observation. The all_dicts tuple should contain the model dictionaries returned from fit_model
    Returns the predicted states.
    """
    # unpack the tuple to get what we need
    unique_states, unique_obs, _, observation_to_index = all_dicts

    # perform a forward fill on the observation in case there are any values in it that we have never seen before
    print(accuracy_score(observation, ffill_obs(observation, unique_obs)))
    
    observation = ffill_obs(observation, unique_obs)
    # get the indices of the observation
    observation_indices = np.array([int(observation_to_index[(o,)]) for o in observation])

    # get the predicted state indices
    _, pred_indices = model.decode(observation_indices.reshape(-1, 1))

    # use the unique_states dictionary to take the indices to the actual states
    pred_states = unique_states[pred_indices, :]

    # return the predicted states
    return pred_states

In [ ]:
# fit the model
num_chords = 1
num_notes = 0
total_songs = 100
song_indices = list(np.arange(total_songs, dtype=int) + 10000)
model, all_dicts = fit_model(train_set, train_lengths, num_chords, num_notes, subset=True, indices=song_indices)

trans_mat = model.transmat_

print(np.argmax(trans_mat, axis=0))
print(np.argmax(trans_mat, axis=1))

# # get a different song observation and the true states
# new_song = load_song_subset(train_set, train_lengths, [len(train_lengths)-1])
# true_states, new_song_obs = dataframe_to_states(new_song, num_chords, num_notes)

# # get the predicted states
# pred_states = predict_states(model, all_dicts, new_song_obs)
# print(pred_states, true_states)

# # get the accuracy
# accuracy = chord_accuracy(pred_states, true_states, num_chords, num_notes)
# accuracy

Processing states: 100%|██████████| 11229/11229 [00:00<00:00, 45433.78it/s]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 16 18 19 20]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16  6 18 19  5]


Other tools:
- dynamax
  - runs on JAX
  - might have a more flexible structure
  - 

In [18]:
num_songs = 5
likelihood, pred_states = model.decode(observation_indices.reshape(-1, 1)[:np.sum(song_lengths[:num_songs])])

# get the actual chords from unique_states
states = unique_states[pred_states, :]

def chord_accuracy(full_pred: np.array, true_states: np.array, num_chords: int=NUM_CHORDS, num_notes: int=NUM_NOTES):
    '''
    Given the predicted matrix of states, compute the misclassification rate compared with the true_observations.
    Could be edited in the future to also compute the accuracy of our predicted note sequence.
    '''
    # obtain the actual predicted chords 
    pred_chords = full_pred[:, num_chords-1]
    true_chords = true_states[:len(pred_chords), num_chords-1]

    # obtain the accuracy
    chord_acc = accuracy_score(true_chords, pred_chords)

    # obtain the actual predicted notes from the state
    if num_notes != 0: 
        pred_notes = full_pred[:, -1]
        true_notes = true_states[:len(pred_notes), -1]

        # obtain the accuracy
        note_acc = accuracy_score(true_notes, pred_notes)
    else:
        note_acc = None
    return chord_acc, note_acc

print(chord_accuracy(states, true_states))

(0.9931506849315068, 0.9726027397260274)
